In [41]:
import csv
import os

if os.environ.get('PWD').startswith("/kaggle"):
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            file_path = os.path.join(dirname, filename)
else:
    file_path = "sudoku.csv"

with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)

In [42]:
import random
BATCH_NUMBER = 1000

def get_sudokus_num(line_number, num):
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)

        # Skip to the desired line
        for _ in range(line_number - 1):
            next(csv_reader)
        lines = []
        # Get the line at line_number
        for _ in range(num):
            lines.append(next(csv_reader))

        # Print the specific line
        return lines

In [43]:
def parse_sudoku(input_string):
    chunk_size = 9
    substrings = [input_string[i:i+chunk_size] for i in range(0, len(input_string), chunk_size)]
    parsed_integers = [[int(char) for char in substring] for substring in substrings]
    return parsed_integers

In [44]:
def get_sudokus(num):
    random_number = random.randint(0, 9000000 - num)
    return get_sudokus_num(random_number, num)

In [45]:
def check_sudoku(arr1, arr2):
    return all(row1 == row2 for row1, row2 in zip(arr1, arr2))

In [46]:
from z3 import Int, Solver, Distinct, And, Sum, If, sat

In [47]:
def setup_algo1():
    # 9x9 matrix of integer variables
    X = [[Int("x_%s_%s" % (i + 1, j + 1)) for j in range(9)] for i in range(9)]

    # each cell contains a value in {1, ..., 9}
    cells_c = [And(1 <= X[i][j], X[i][j] <= 9) for i in range(9) for j in range(9)]

    # each row contains a digit at most once
    rows_c = [Distinct(X[i]) for i in range(9)]

    # each column contains a digit at most once
    cols_c = [Distinct([X[i][j] for i in range(9)]) for j in range(9)]

    # each 3x3 square contains a digit at most once
    sq_c = [
        Distinct([X[3 * i0 + i][3 * j0 + j] for i in range(3) for j in range(3)])
        for i0 in range(3)
        for j0 in range(3)
    ]

    sudoku_c = cells_c + rows_c + cols_c + sq_c
    return sudoku_c,X

In [48]:
def setup_algo2():
    X = [[Int(f"a[{i}][{j}]") for i in range(9)] for j in range(9)]

    # each cell contains a value in {1, ..., 9}
    cells_c = [And(1 <= X[i][j], X[i][j] <= 9) for i in range(9) for j in range(9)]

    sum_h = [Sum([X[i][j] for i in range(9)]) == 45 for j in range(9)]
    sum_v = [Sum([X[j][i] for i in range(9)]) == 45 for j in range(9)]


    rows_c = [Distinct(X[j]) for j in range(9)]
    cols_c = [Distinct([X[i][j] for i in range(9)]) for j in range(9)]
    sq_c = [
        Distinct([X[3 * i0 + i][3 * j0 + j] for i in range(3) for j in range(3)])
        for i0 in range(3)
        for j0 in range(3)
    ]


    sudoku_c = cells_c + sum_h + sum_v + rows_c + cols_c + sq_c
    return sudoku_c,X

In [49]:
def test_algo(sudoku_c, X, instance, solution):
    instance_c = [
        If(instance[i][j] == 0, True, X[i][j] == instance[i][j])
        for i in range(9)
        for j in range(9)
    ]

    s = Solver()
    s.add(sudoku_c + instance_c)
    if s.check() == sat:
        m = s.model()
        r = [[m.evaluate(X[i][j]) for j in range(9)] for i in range(9)]
        # print(r, solution)
        return check_sudoku(r, solution)
    else:
        return False

In [50]:
def full_test(sudokus_to_test, setup_fn):
    solved = []
    sudoku_c, X = setup_fn()
    for one_sudoku in sudokus_to_test:
        current_sudoku = parse_sudoku(one_sudoku[0])
        answer = parse_sudoku(one_sudoku[1])
        solved.append(test_algo(sudoku_c, X, current_sudoku, answer))
    return solved.count(True)

In [51]:
def test(sudokus, setup_algo):
    t = full_test(sudokus, setup_algo)
    if t == len(sudokus):
        print("Test passed")
    else:
        print("Test failed", t)

In [52]:
import timeit

In [53]:
BATCH_NUMBER = 10
s = get_sudokus(BATCH_NUMBER)

In [54]:
test(s, setup_algo1)

Test passed


In [55]:
print(timeit.timeit("test(s, setup_algo1)", globals=locals(), number=1))

Test passed
0.4285832160003338


In [56]:
print(timeit.timeit("test(s, setup_algo2)", globals=locals(), number=1))

Test passed
0.5237615519999963


In [57]:
def simple_validy_check(board):
    # Check rows
    for row in board:
        if len(set(row)) != 9:
            return False

    # Check columns
    for col in range(9):
        column = [board[row][col] for row in range(9)]
        if len(set(column)) != 9:
            return False

    # Check 3x3 subgrids
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            square = [board[row][col] for row in range(i, i+3) for col in range(j, j+3)]
            if len(set(square)) != 9:
                return False

    return True

In [58]:
def validate(board):
    # this function is wrong and do not validate every sudoku !!!
    for row in board:
        if sum(row) != 45:
            return False

    for col in range(9):
        column = [board[row][col] for row in range(9)]
        if sum(column) != 45:
            return False


    if len(set(board[0])) != 9:
        return False

    for i in range(1, 9):
        if board[0][i] < 1 and board[0][i] > 9:
            return False

    return True

In [59]:
def validation_test_1(sudokus_to_test):
    solved = []
    for one_sudoku in sudokus_to_test:
        answer = parse_sudoku(one_sudoku[1])
        solved.append(simple_validy_check(answer))
    return solved.count(True)

def validation_test_2(sudokus_to_test):
    solved = []
    for one_sudoku in sudokus_to_test:
        answer = parse_sudoku(one_sudoku[1])
        solved.append(validate(answer))
    return solved.count(True)

In [60]:
BATCH_NUMBER = 1000
s = get_sudokus(BATCH_NUMBER)
print(timeit.timeit("print(validation_test_1(s))", globals=locals(), number=1))
print(timeit.timeit("print(validation_test_2(s))", globals=locals(), number=1))
# note: second function is wrong

1000
0.02522273700014921
1000
0.01439215900063573
